In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

!pip install mlxtend

/kaggle/input/arl-data/armut_data.csv


In [2]:
############################################
# ASSOCIATION RULE LEARNING (BİRLİKTELİK KURALI ÖĞRENİMİ)
############################################
# 1. Veri Ön İşleme
# 2. ARL Veri Yapısını Hazırlama (Invoice-Product Matrix)
# 3. Birliktelik Kurallarının Çıkarılması
# 4. Çalışmanın Scriptini Hazırlama
# 5. Sepet Aşamasındaki Kullanıcılara Ürün Önerisinde Bulunmak

# Görev 1: Veriyi Hazırlama
# Görev 2: Birliktelik Kuralları Üretiniz ve Öneride bulununuz

In [4]:
############################################
# 1. Veri Ön İşleme
############################################

# !pip install mlxtend
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
# çıktının tek bir satırda olmasını sağlar.
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

In [5]:
# Görev 1: Veriyi Hazırlama
# Adım 1: armut_data.csv dosyasını okutunuz.

df_ = pd.read_csv("/kaggle/input/arl-data/armut_data.csv")
df = df_.copy()

In [6]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [6]:
df.dtypes

UserId         int64
ServiceId      int64
CategoryId     int64
CreateDate    object
dtype: object

In [7]:
# Adım 2: ServisID her bir CategoryID özelinde farklı bir hizmeti temsil etmektedir. ServiceID ve CategoryID’yi "_" ile birleştirerek bu hizmetleri
# temsil edecek yeni bir değişken oluşturunuz
df["ServiceId"] = df["ServiceId"].astype(str)
df["CategoryId"] = df["CategoryId"].astype(str)
df["Hizmet"] = df["ServiceId"] + "_" + df["CategoryId"]

In [8]:
# Adım 3: Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır, herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır. Association Rule
# Learning uygulayabilmek için bir sepet (fatura vb.) tanımı oluşturulması gerekmektedir. Burada sepet tanımı her bir müşterinin aylık aldığı
# hizmetlerdir. Örneğin; 7256 id'li müşteri 2017'in 8.ayında aldığı 9_4, 46_4 hizmetleri bir sepeti; 2017’in 10.ayında aldığı 9_4, 38_4 hizmetleri
# başka bir sepeti ifade etmektedir. Sepetleri unique bir ID ile tanımlanması gerekmektedir. Bunun için öncelikle sadece yıl ve ay içeren yeni bir
# date değişkeni oluşturunuz. UserID ve yeni oluşturduğunuz date değişkenini "_" ile birleştirirek ID adında yeni bir değişkene atayınız. 

df["CreateDate"] = pd.to_datetime(df["CreateDate"])

df["New_Date"] = df["CreateDate"].dt.to_period("M")
df.head()

df["UserId"] = df["UserId"].astype("str")
df["New_Date"] = df["New_Date"].astype("str")
df["SepetID"] = df["UserId"] + "_" + df["New_Date"]
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date,SepetID
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08


In [9]:
# Görev 2: Birliktelik Kuralları Üretiniz ve Öneride bulununuz

# Adım 1: Aşağıdaki gibi sepet, hizmet pivot table’i oluşturunuz.

df_pivot = df.pivot_table(index="SepetID", columns="Hizmet", aggfunc=lambda x: 1, fill_value=0)

In [10]:
df_pivot.head()

CategoryId                                                                                                                                                                                                                                                    CreateDate                                                                                                                                                                                                                                                    New_Date                                                                                                                                                                                                                                                    ServiceId                                                                                                                                                                                                                                                    UserId                                                                                                                                                                                                                                                   
Hizmet               0_8 10_9 11_11 12_7 13_11 14_7 15_1 16_8 17_5 18_4 19_6 1_4 20_5 21_5 22_0 23_10 24_10 25_0 26_7 27_7 28_4 29_0 2_0 30_2 31_6 32_4 33_4 34_6 35_11 36_1 37_0 38_4 39_10 3_5 40_8 41_3 42_1 43_2 44_0 45_6 46_4 47_7 48_5 49_1 4_5 5_11 6_7 7_3 8_5 9_4        0_8 10_9 11_11 12_7 13_11 14_7 15_1 16_8 17_5 18_4 19_6 1_4 20_5 21_5 22_0 23_10 24_10 25_0 26_7 27_7 28_4 29_0 2_0 30_2 31_6 32_4 33_4 34_6 35_11 36_1 37_0 38_4 39_10 3_5 40_8 41_3 42_1 43_2 44_0 45_6 46_4 47_7 48_5 49_1 4_5 5_11 6_7 7_3 8_5 9_4      0_8 10_9 11_11 12_7 13_11 14_7 15_1 16_8 17_5 18_4 19_6 1_4 20_5 21_5 22_0 23_10 24_10 25_0 26_7 27_7 28_4 29_0 2_0 30_2 31_6 32_4 33_4 34_6 35_11 36_1 37_0 38_4 39_10 3_5 40_8 41_3 42_1 43_2 44_0 45_6 46_4 47_7 48_5 49_1 4_5 5_11 6_7 7_3 8_5 9_4       0_8 10_9 11_11 12_7 13_11 14_7 15_1 16_8 17_5 18_4 19_6 1_4 20_5 21_5 22_0 23_10 24_10 25_0 26_7 27_7 28_4 29_0 2_0 30_2 31_6 32_4 33_4 34_6 35_11 36_1 37_0 38_4 39_10 3_5 40_8 41_3 42_1 43_2 44_0 45_6 46_4 47_7 48_5 49_1 4_5 5_11 6_7 7_3 8_5 9_4    0_8 10_9 11_11 12_7 13_11 14_7 15_1 16_8 17_5 18_4 19_6 1_4 20_5 21_5 22_0 23_10 24_10 25_0 26_7 27_7 28_4 29_0 2_0 30_2 31_6 32_4 33_4 34_6 35_11 36_1 37_0 38_4 39_10 3_5 40_8 41_3 42_1 43_2 44_0 45_6 46_4 47_7 48_5 49_1 4_5 5_11 6_7 7_3 8_5 9_4
SepetID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
0_2017-08              0    0     0    0     0    0    0    0    0    0    0   0    0    0    0     0     0    0    0    0    0    0   0    0    0    0    0    0     0    0    0    0 

In [10]:
df_pivot = df_pivot.astype(bool)
df_pivot.head()

CategoryId                                                                                                                                                                                                                                                                                                                                                        CreateDate                                                                                                                                                                                                                                                                                                                                                        New_Date                                                                                                                                                                                                                                                                                                                                                        ServiceId                                                                                                                                                                                                                                                                                                                                                        UserId                                                                                                                                                                                                                                                                                                                                                       
Hizmet               0_8   10_9  11_11   12_7  13_11   14_7   15_1   16_8   17_5   18_4   19_6    1_4   20_5   21_5   22_0  23_10  24_10   25_0   26_7   27_7   28_4   29_0    2_0   30_2   31_6   32_4   33_4   34_6  35_11   36_1   37_0   38_4  39_10    3_5   40_8   41_3   42_1   43_2   44_0   45_6   46_4   47_7   48_5   49_1    4_5   5_11    6_7    7_3    8_5    9_4        0_8   10_9  11_11   12_7  13_11   14_7   15_1   16_8   17_5   18_4   19_6    1_4   20_5   21_5   22_0  23_10  24_10   25_0   26_7   27_7   28_4   29_0    2_0   30_2   31_6   32_4   33_4   34_6  35_11   36_1   37_0   38_4  39_10    3_5   40_8   41_3   42_1   43_2   44_0   45_6   46_4   47_7   48_5   49_1    4_5   5_11    6_7    7_3    8_5    9_4      0_8   10_9  11_11   12_7  13_11   14_7   15_1   16_8   17_5   18_4   19_6    1_4   20_5   21_5   22_0  23_10  24_10   25_0   26_7   27_7   28_4   29_0    2_0   30_2   31_6   32_4   33_4   34_6  35_11   36_1   37_0   38_4  39_10    3_5   40_8   41_3   42_1   43_2   44_0   45_6   46_4   47_7   48_5   49_1    4_5   5_11    6_7    7_3    8_5    9_4       0_8   10_9  11_11   12_7  13_11   14_7   15_1   16_8   17_5   18_4   19_6    1_4   20_5   21_5   22_0  23_10  24_10   25_0   26_7   27_7   28_4   29_0    2_0   30_2   31_6   32_4   33_4   34_6  35_11   36_1   37_0   38_4  39_10    3_5   40_8   41_3   42_1   43_2   44_0   45_6   46_4   47_7   48_5   49_1    4_5   5_11    6_7    7_3    8_5    9_4    0_8   10_9  11_11   12_7  13_11   14_7   15_1   16_8   17_5   18_4   19_6    1_4   20_5   21_5   22_0  23_10  24_10   25_0   26_7   27_7   28_4   29_0    2_0   30_2   31_6   32_4   33_4   34_6  35_11   36_1   37_0   38_4  39_10    3_5   40_8   41_3   42_1   43_2   44_0   45_6   46_4   47_7   48_5   49_1    4_5   5_11    6_7    7_3    8_5    9_4
SepetID                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [11]:
# Adım 2: Birliktelik kurallarını oluşturunuz.

frequent_itemsets = apriori(df_pivot,
                           min_support=0.05,
                           use_colnames=False)

In [12]:
frequent_itemsets.shape

(186, 2)

In [13]:
frequent_itemsets.sort_values("support", ascending=False)

,support,itemsets
38,0.238121,"(9, 59)"
134,0.238121,"(209, 59, 159)"
133,0.238121,"(209, 59, 109)"
132,0.238121,"(59, 109, 159)"
77,0.238121,"(209, 109)"
...,...,...
128,0.056627,"(154, 204, 54)"
144,0.056627,"(104, 154, 204)"
150,0.056627,"(104, 154, 4, 54)"
151,0.056627,"(104, 204, 4, 54)"


In [15]:
rules = association_rules(frequent_itemsets,
                         metric="support",
                         min_threshold=0.001)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(4),(54),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
1,(54),(4),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
2,(104),(4),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
3,(4),(104),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
4,(154),(4),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
1075,(193),"(43, 243, 93, 143)",0.067762,0.067762,0.067762,1.0,14.757563,0.063170,inf,1.0
1076,(43),"(193, 243, 93, 143)",0.067762,0.067762,0.067762,1.0,14.757563,0.063170,inf,1.0
1077,(143),"(193, 43, 93, 243)",0.067762,0.067762,0.067762,1.0,14.757563,0.063170,inf,1.0
1078,(243),"(193, 43, 93, 143)",0.067762,0.067762,0.067762,1.0,14.757563,0.063170,inf,1.0


In [16]:
rules[(rules["support"]>0.045) & (rules["confidence"]>0.1) & (rules["lift"]>5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(4),(54),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
1,(54),(4),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
2,(104),(4),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
3,(4),(104),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
4,(154),(4),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
1075,(193),"(43, 243, 93, 143)",0.067762,0.067762,0.067762,1.0,14.757563,0.063170,inf,1.0
1076,(43),"(193, 243, 93, 143)",0.067762,0.067762,0.067762,1.0,14.757563,0.063170,inf,1.0
1077,(143),"(193, 43, 93, 243)",0.067762,0.067762,0.067762,1.0,14.757563,0.063170,inf,1.0
1078,(243),"(193, 43, 93, 143)",0.067762,0.067762,0.067762,1.0,14.757563,0.063170,inf,1.0


In [18]:
rules[(rules["support"] > 0.05) & (rules["confidence"] > 0.5) & (rules["lift"] > 5)]. \
      sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(4),(54),0.056627,0.056627,0.056627,1.0,17.659311,0.053421,inf,1.0
699,"(222, 22)","(122, 172)",0.130286,0.130286,0.130286,1.0,7.675396,0.113312,inf,1.0
729,(81),"(131, 31, 231)",0.066568,0.066568,0.066568,1.0,15.022147,0.062137,inf,1.0
730,(131),"(81, 31, 231)",0.066568,0.066568,0.066568,1.0,15.022147,0.062137,inf,1.0
731,(31),"(81, 131, 231)",0.066568,0.066568,0.066568,1.0,15.022147,0.062137,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
359,(156),"(56, 106)",0.120963,0.120963,0.120963,1.0,8.266976,0.106331,inf,1.0
360,"(56, 106)",(206),0.120963,0.120963,0.120963,1.0,8.266976,0.106331,inf,1.0
361,"(56, 206)",(106),0.120963,0.120963,0.120963,1.0,8.266976,0.106331,inf,1.0
362,"(106, 206)",(56),0.120963,0.120963,0.120963,1.0,8.266976,0.106331,inf,1.0


In [29]:
# Adım 3: arl_recommender fonksiyonunu kullanarak en son 4_0 hizmetini alan bir kullanıcıya hizmet önerisinde bulununuz.
product_id = 4

sorted_rules = rules.sort_values("lift", ascending=False)

recommendation_list = []

for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])
            

recommendation_list[0:3]

[54, 104, 154]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
